In [160]:
%load_ext autoreload
%autoreload 2

# Relatorio - Sharpe Optimization

Este arquivo tem como objetivo docuemntar os passos feitos para otimizar o indice Sharpe de um portfolio tentando diminuir o efeito do bias de sobrevivencia 

## Obtendo dados da Bloomberg

Os .CSVs obtidos por meio do terminal da Bloomberg foram salvos na pasta dataset/stocks e unidos em um unico DataFrame

In [116]:
import os 
import pandas as pd 
from copy import deepcopy

def merge_csv_to_df():
    """ Une os CSV's de ações na pasta para um mesmo DataFrame
    
    Returns:
        DataFrame -- Pandas Dataframe com os dados
    """ 
    list_of_csvs = os.listdir(os.path.join('dataset','stocks'))
    first = True
    
    # Checa todos arquivos de CSV
    for ticker in list_of_csvs:

        company_data = pd.read_csv(os.path.join('dataset','stocks', ticker)).iloc[2:]
        if company_data.empty:
            pass
        
        else:
            company_data.index = company_data.iloc[:,0]

    #         print(company_data)
            company_data = company_data.iloc[:,1]
            company_data = pd.DataFrame(company_data)
            ticker = ticker[:-4]
            company_data.columns = [ticker]
            # Na primeira iteração cria a variável prices
            if (first == True):
                prices = deepcopy(company_data)
                first = False

            # Se contém preços, salva no arquivo prices a coluna com o nome da ação.
            else:
                prices = prices.merge(right = company_data, how='outer', left_on = 'ticker', right_on = 'ticker')
      
    print("Stock market CSV's merged!")
    prices = prices[:-2]
    return prices.astype(float)

prices_bloomberg = merge_csv_to_df()
prices_bloomberg = prices_bloomberg.astype(float)

Stock market CSV's merged!


## Obtendo dados da Brasil Plural

Os dados da Brasil Plural foram obtidos por meio do banco de dados e adaptados para o uso da biblioteca Quantin

In [117]:
import pandas as pd
import os 

prices_brasilplural = pd.read_csv(os.path.join('dataset','stocks_brasil_plural.csv'))
tickers_brasilplural = prices_brasilplural.symbol.unique()
prices_brasilplural = prices_brasilplural.pivot(index='date',columns='symbol',values='close')

## Obtendo dados da Nasdaq

Os dados da Nasdaq foram obtidos por meio de sua API e salvos em um .csv

In [118]:
prices_nasdaq = pd.read_csv(os.path.join('dataset','stocks_nasdaq.csv'))
prices_nasdaq = prices_nasdaq.drodpna(axis = 0)
prices_nasdaq.reset_index(drop = True, inplace = True)
prices_nasdaq = prices_nasdaq.iloc[:,1:]

/home/tydeg/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



## Comparando as bases

O codigo abaixo comparou a bases da Nasdaq, Brasil Plural e Bloomberg.

In [184]:
from datetime import datetime
import plotly.graph_objects as go

def compare_data (symbol):
    # Organizar os dados da Nasdaq 
    n = prices_nasdaq[prices_nasdaq['symbol'] == symbol][['date', 'close']]
    n['date'] = pd.to_datetime(n['date'], format= '%m/%d/%Y')
    n.columns = ['date', 'close_nasdaq']
    # Organizar os dados da Brasil Plural
    bp = prices_brasilplural.loc[:,symbol].to_frame()
    bp.index.names = ['date']
    bp.index = pd.to_datetime(bp.index, format= '%Y-%m-%d')
    bp.columns = ['close_brasilplural']    

    df = bp.merge(n, left_on = 'date', right_on = 'date', how = 'inner' )
    df.sort_values('date', inplace= True)
    df.reset_index(drop = True)

    fig = go.Figure()
    not_in_bloomberg = symbol
    
    # Organizar os dados da Bloomberg se existirem
    if symbol.replace(" ", "") in prices_bloomberg.columns:
        bl = prices_bloomberg.loc[:,symbol].to_frame()
        bl.index.names = ['date']
        bl.index = pd.to_datetime(bl.index, format= '%Y-%m-%d')
        bl.columns = ['close_bloomberg']    
        df = df.merge(bl, left_on = 'date', right_on = 'date', how = 'inner' )
        df['close_bloomberg'] = ((df['close_bloomberg']).pct_change()+1).cumprod()

        fig.add_trace(go.Scatter(x=df['date'], 
                         y=df['close_bloomberg'],
                         text = 'Bloomberg',
                         mode='lines',
                         name='lines'))
        not_in_bloomberg = False

        
     
    df = df[df['date'] < datetime.strptime('2018-11-01', "%Y-%m-%d")]
    df['close_brasilplural'] = ((df['close_brasilplural']).pct_change()+1).cumprod()
    df['close_nasdaq'] = ((df['close_nasdaq']).pct_change()+1).cumprod()

    # Plotar os graficos
    fig.add_trace(go.Scatter(x=df['date'], 
                         y=df['close_brasilplural'],
                         text = 'Brasil Plural',
                         mode='lines',
                         name='lines'))
    fig.add_trace(go.Scatter(x=df['date'], 
                         y= df['close_nasdaq'],
                         text = 'Nasdaq',
                         mode='lines',
                         name='lines'))    
    fig.update_layout(
    title=symbol)
    
    empty_df = False
    if df.empty:
        empty_df = symbol
    else:
        fig.show()

    return df, not_in_bloomberg, empty_df

not_in_bloomberg_list = []
empty_df_list = []
# for i in tickers_brasilplural[450:500]:
#     df, not_in_bloomberg, empty_df = compare_data(i)
#     empty_df_list.append(empty_df) 
#     not_in_bloomberg_list.append(not_in_bloomberg)

Analisando os gráficos, chegamos à conclusão de que os gráficos da Bloomberg assim como os da Nasdaq não estão ajustados de acordo com os dividendos. Dessa forma, foi feito o seguinte: 
As ações que estão na base da Brasil Plural (Ajustadas por dividendo) serão utilizadas na análise. As outras ações que não estão no SP500 e estão na base da Bloomberg (Não ajustadas por dividendo) serão unidas a essa base inicial. Os resultados obtidos a partir dos modelos criados por essa base estarão subestimados, mas é que está ao nosso alcance no momento.
Além disso foram constatados os seguintes fatos:
* Ações que não estão no Bloomberg: BBT,BF B,BHGE,BRK B,CBS,HCP,JEC,KIM,SYMC
* Ações não encontradas: AJG,ALL, AMD, ATVI, BBT, BF B, BHGE, BRK B, CBS, CELG, ESRX, FE, FOX, FOXA, FRT, HCP, HP, JEC, LLL, LMT, NFX, RHT, SCG, STI, SYMC, TGT, TSCO, TSS, UNM, VIAB, WELL, ZTS
* Ações erradas na Brasil Plural: ARNC, DOV
* Ações erradas na Nasdaq: HLT
* Ações erradas na Bloomberg: KLAC



In [119]:
for t in tickers_brasilplural:
    black_list = ['ARNC', 'DOV']
    if t in black_list:
        pass
    else:
        prices_bloomberg[t] = prices_brasilplural[t]
        
returns_bloomberg = prices_bloomberg.pct_change(fill_method = None)

## Organizando os retornos da Bloomberg

O codigo abaixo buscou alterar a tabela de retorno da otimizacao para respeitar as acoes do indice SP500 vigente na epoca. Este indice muda de composicao em marco ou brasil do ano, e em setembro ou outubro. Entao por exemplo, em agosto de 2018 as acoes disponiveis para otmizacao devem ser aquelas divulgadas em abril de 2018. O procedimento foi documentado no codigo abaixo:

In [126]:
import os 
import datetime
import xlrd
import collections 

# Foi criado um dicionario com composicoes.
# Este dicionario possui como key o mes e ano em que a composicao foi divulgada
# E como valores, uma lista com os tickers.
# Os dados utilizados foram obtidos pelo Gustavo no terminal da Bloomberg
compositions = dict()
for file_name in  os.listdir(os.path.join('dataset', 'sp500 composition')):
    day = (file_name[14:16]) 
    month_str = (file_name[10:13]) 
    year = (file_name[17:21])
    if(month_str == 'Apr'):
        month = 4
    if(month_str == 'Oct'):
        month = 10
    if(month_str == 'Sep'):
        month = 9
    if(month_str == 'Mar'):
        month = 3   
    workbook = xlrd.open_workbook(os.path.join('dataset', 'sp500 composition', file_name), on_demand = True)
    worksheet = workbook.sheet_by_index(0)
    first_row = []
    data = []
    for row in range(1, worksheet.nrows):
        elm = {}
        data.append(worksheet.cell_value(row,0).split(' U')[0].replace('/',' '))
    key = month_str + '/' + year
    compositions[datetime.datetime(int(year), month,1)] = data #.strftime('%Y-%m-%d')

compositions = collections.OrderedDict(sorted(compositions.items()))

In [185]:
import datetime
import dateutil.relativedelta

# As keys do dicionario foram salvas numa lista chamada dates
dates = []
for k in compositions:
    dates.append(k)

# A lista date foi iterada, e para cada data de divulgacao foram pegos os meses do seu Lower Bound
date_grid_aux = []
for n, d in enumerate(dates):
    from_date = (d - dateutil.relativedelta.relativedelta(months = 11 ))
    date_grid_aux.append([from_date,d])

# Foi criada outra lista, contendo tambem o upper bound de cada composicao de carteira 
# No caso o upper bound eh a proxima divulgacao de carteira
date_grid = []
for n, dg in enumerate(date_grid_aux):
    if (n == len(date_grid_aux) - 1):
        from_date = date_grid_aux[n][0]
        from_date = from_date.strftime('%Y-%m-%d')
        date_grid.append([date_grid_aux[n][1].strftime('%Y-%m-%d'),from_date, None]) 
    else:
        from_date = date_grid_aux[n][0]
        from_date = from_date.strftime('%Y-%m-%d')
        to_date = date_grid_aux[n + 1][1] + dateutil.relativedelta.relativedelta(months = 1 )
        to_date = to_date.strftime('%Y-%m-%d')
        date_grid.append([date_grid_aux[n][1].strftime('%Y-%m-%d'), from_date, to_date]) 
        
# Este grid de data inicial e final foi iterada, salvando em uma lista partes do DF que contem as 
# acoes para cada periodo
dataframe_list = []
for n,dg in enumerate(date_grid):
    if (n == len(date_grid_aux) - 1):
        dataframe_slice = deepcopy(returns_bloomberg.loc[ (returns_bloomberg.index >= dg[1]),stocks_from_period] )
        dataframe_list.append(dataframe_slice)
    else:
        stocks_from_period = [i for i in compositions[datetime.datetime.strptime(dg[0], '%Y-%m-%d')] if i in df.columns]
        dataframe_slice = deepcopy(returns_bloomberg.loc[ (returns_bloomberg.index >= dg[1]) & (returns_bloomberg.index < dg[2]),stocks_from_period] )
        dataframe_list.append(dataframe_slice)
        
# Foi criado um dataframe vazio com os mesmo indices e colunas da tabela de retorno original
# Essa tabela foi preenchida com os pedacos com as acoes de cada periodo.
import numpy as np
returns = pd.DataFrame(np.nan, index=returns_bloomberg.index, columns=returns_bloomberg.columns)

for i in dataframe_list:
    returns = returns.fillna(i)

Abaixo os retornos e o benchmark foram salvos no pickle data_input.pickle

In [201]:
from quantfin.datawrangling import merge_csv_to_df, get_sp500, get_tyde, save_to_pickle,  return_from_pickle
import os 
from quantfin.strategy import Benchmark
from quantfin.datawrangling import save_to_pickle
import pandas as pd 
benchmark_tyde = get_tyde()
benchmark_sp500 = get_sp500()

b_tyde = Benchmark(benchmark_tyde, name = 'Tyde')
b_sp500 = Benchmark(benchmark_sp500, name = 'SP 500')
save_to_pickle(path = os.path.join('dataset',"processed", "data_input.pickle"),
               variables = [returns, b_sp500, b_tyde]
              )

[2019-12-26 09:57:21] Tyde returns obtained 
[2019-12-26 09:57:23] S&P 500 Returns obtained 
[2019-12-26 09:57:24] Variables saved in path dataset/processed/data_input.pickle 


In [5]:
import os 

returns, b_sp500, b_tyde = return_from_pickle(path = os.path.join('dataset',"processed", "data_input.pickle") )

Abaixo foram testados os retornos com diferentes pesos maximos por acao. Foram criados 4 portfolios, diferindo pelo peso maximo permitido por acao: 1.67%, 2.5%, 5% e 10%. Os portfolios foram salvos na pasta portfolios. 

In [2]:
from quantfin.strategy import Strategy 
from quantfin.datawrangling import save_to_pickle
tolerance_list = [0.01]
max_w_list = [0.0167,0.025,0.05,0.1]
time_os_list = [1]

count = 0
for tolerance in tolerance_list:
    for max_w in max_w_list:
        for time_os in time_os_list:
            portfolio = Strategy(method = 'max_sharpe',
                            name = 'Max Sharpe {}'.format(str(count)),
                            log_path = 'logs/max_sharpe{}.csv'.format(str(count)),
                            rets = returns,
                            time_is = 12, 
                            time_os = time_os, 
                            max_w= max_w,
                            date_from = '2009-01-01', 
                            date_to = '2018-11-30',
                            threads= 12,
                            minimization_tolerance= tolerance)
            save_to_pickle('portfolios/max_sharpe{}.pickle'.format(str(count)), portfolio)
            count = count + 1 

In [9]:
import pandas as pd 
from quantfin.datawrangling import return_from_pickle
from quantfin.analysis import compare_strategies

index_list = [0,1,2,3]
portfolio_list = []
max_w_list = [0.0167,0.025,0.05,0.1]
for i in index_list:
    diff =  data['date'].iloc[-1] - data['date'][0] 
    portfolio = return_from_pickle('portfolios/max_sharpe{}.pickle'.format(i))
    portfolio.name = 'Max Sharpe ' + str(max_w_list[i]) 
    portfolio_list.append(portfolio)
    
compare_strategies(portfolio_list + [b_tyde],b_sp500 )

,Annaul Return,Annual Volatility,Beta,Sharpe,Sortino,Treynor,Starr,Kurtosis,Skew,CVar 5%,Var 5%,Maximum Drawdown,Avg. Turnover:,avg. Size:
Strategy,,,,,,,,,,,,,,
Max Sharpe 0.0167,9.76%,12.79%,0.90,0.76,0.06,0.11,-488.15%,2.64,-0.64,-2.00%,-1.37%,10.69%,27.24%,64
Max Sharpe 0.025,9.09%,13.02%,0.89,0.70,0.06,0.10,-444.43%,2.60,-0.65,-2.04%,-1.43%,12.45%,31.03%,50
Max Sharpe 0.05,9.17%,13.72%,0.89,0.67,0.05,0.10,-428.60%,2.18,-0.64,-2.14%,-1.45%,14.65%,35.72%,32
Max Sharpe 0.1,6.91%,14.70%,0.89,0.47,0.04,0.08,-299.32%,2.10,-0.65,-2.31%,-1.57%,19.99%,39.35%,19
Tyde,15.15%,16.48%,1.16,0.92,0.07,0.13,-573.31%,2.28,-0.66,-2.64%,-1.82%,18.31%,0.00%,0
SP 500,8.38%,12.80%,1.00,0.65,0.05,0.08,-412.18%,3.20,-0.54,-2.03%,-1.36%,14.16%,0.00%,0
